In [1]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [3]:
COLLECTION_NAME = 'data_true_6'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[156748727,
 792192261341663232,
 967231652932993024,
 1125059782417629185,
 1353184880469794817,
 1289439912194789376,
 858731841674633216,
 1300501588725739520,
 1094736215784460288,
 1274478236022345728,
 1316769175897960452,
 1299544734495469568,
 223057447,
 916830035503394817,
 1113853698344513537,
 1282426370996539392,
 710300004888358916,
 1270690068911423488,
 1367453941328347142,
 851634708286853120,
 26110899,
 1274035734295597058,
 1124937623246479361,
 1310231325560000518,
 3279188756,
 1128005011865104384,
 1370196435614961674,
 977417186531729413,
 1282639248764448768,
 41416662,
 1340194207839920128,
 1247972611587543047,
 1322157700986273793,
 100736799,
 1021919008856199169,
 1103603778438127618,
 1219439539405672450,
 624608605,
 960823509352988672,
 1103946485048082433,
 1350397855941292037,
 1663758864,
 64285786,
 1002353869593051136,
 1353184880469794817,
 1574088482,
 1168301267258683392,
 889277329721233408,
 1332653411209187329,
 1010020749652811777,
 12926511

In [19]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 849800022094880768})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 849800022094880768})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 849800022094880768})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 849800022094880768})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 849800022094880768})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{849800022094880768}
849800022094880768 Tue Apr 20 23:23:07 +0000 2021
849800022094880768
849800022094880768
Wed Apr 21 05:07:35 +0000 2021
20668.0
Wed Apr 21 14:07:29 +0000 2021
53062.0
Wed Apr 21 06:06:18 +0000 2021
24191.0
Wed Apr 21 08:28:17 +0000 2021
32710.0
Tue Apr 20 23:23:18 +0000 2021
11.0
Fri Apr 30 22:12:19 +0000 2021
859752.0
Wed Apr 21 10:53:16 +0000 2021
41409.0
Wed Apr 21 03:31:36 +0000 2021
14909.0
Tue Apr 20 23:24:04 +0000 2021
57.0
Wed Apr 21 03:33:04 +0000 2021
14997.0
Wed Apr 21 12:04:06 +0000 2021
45659.0
Wed Apr 21 08:44:51 +0000 2021
33704.0
Wed Apr 21 10:56:33 +0000 2021
41606.0
Tue Apr 20 23:34:07 +0000 2021
660.0
Wed Apr 21 03:33:53 +0000 2021
15046.0
Wed Apr 21 05:55:40 +0000 2021
23553.0
Wed Apr 21 08:03:13 +0000 2021
31206.0
Sat Apr 24 04:59:26 +0000 2021
279379.0
Tue Apr 20 23:33:00 +0000 2021
593.0
 user_id: 849800022094880768, created_at: Tue Apr 20 23:23:07 +0000 2021, difference_seconds: 0
├──  user_id: 1227156130066886657, created_at: Wed Apr 21 05:0

In [20]:
remove_duplicates(root)

0
0


In [21]:
for i in range(2,10):
    add_depth_nodes(root, i, tweet_user_ids, db, COLLECTION_NAME)
    remove_duplicates(root)

[Node('/849800022094880768/1227156130066886657', created_at='Wed Apr 21 05:07:35 +0000 2021', difference_seconds=20668.0), Node('/849800022094880768/1275057204660326401', created_at='Wed Apr 21 14:07:29 +0000 2021', difference_seconds=53062.0), Node('/849800022094880768/3978029200', created_at='Wed Apr 21 06:06:18 +0000 2021', difference_seconds=24191.0), Node('/849800022094880768/1314882902883209222', created_at='Wed Apr 21 08:28:17 +0000 2021', difference_seconds=32710.0), Node('/849800022094880768/2826887982', created_at='Tue Apr 20 23:23:18 +0000 2021', difference_seconds=11.0)]
 user_id: 849800022094880768, diff_s: 0
├──  user_id: 1227156130066886657, diff_s: 20668.0
├──  user_id: 1275057204660326401, diff_s: 53062.0
├──  user_id: 3978029200, diff_s: 24191.0
├──  user_id: 1314882902883209222, diff_s: 32710.0
├──  user_id: 2826887982, diff_s: 11.0
├──  user_id: 1364205162928160770, diff_s: 859752.0
├──  user_id: 1238662677250977792, diff_s: 41409.0
├──  user_id: 1153265001706094594

In [22]:
# export tree
from anytree.exporter import JsonExporter, DictExporter
exporter = DictExporter()
data = exporter.export(root)
with open(COLLECTION_NAME + '_tree.json', 'w') as fp:
    json.dump(data, fp)    

In [5]:
users= [14330880, 20402945, 3191437439, 1147804652039069696, 1329524222109097995, 14500743, 1282297722364727296, 9633802, 1052626822645596161, 25589776, 17710740, 215863060, 226024854, 1333137558893719552, 41085467, 624413, 1143770884974399488, 1295043616742805506, 295860511, 353701419, 1233104313771184129, 768552674719567876, 15108530, 17525171, 48310197, 1278251649803251713, 17852343, 19898168, 133663801, 20583993, 158865339, 472122299, 1652541, 18774524, 980924206132195328, 849800022094880768, 1371529262566608908, 63485764, 356631237, 1283802294186848257, 149516743, 96877126, 21694413, 19407053, 14946895, 16396368, 1276154115773599744, 7313362, 2987385810, 1232689061049126912, 596792665, 786324746719924224, 1260307716385443840, 1310782718603456513, 2735591, 488554346, 26329195, 23922797, 2291106157, 1303630426234408961, 15969142, 19013879, 1016487483138138112, 145802745, 17121146, 1291779641519169536, 14790524, 14594813, 1381246086363365376]

In [6]:
for user in users:
    print(user, get_followers_len(user, db))

14330880 0
20402945 0
3191437439 0
1147804652039069696 0
1329524222109097995 0
14500743 0
1282297722364727296 2355
9633802 0
1052626822645596161 0
25589776 0
17710740 0
215863060 0
226024854 0
1333137558893719552 0
41085467 0
624413 0
1143770884974399488 629
1295043616742805506 0
295860511 0
353701419 0
1233104313771184129 0
768552674719567876 0
15108530 0
17525171 0
48310197 9896
1278251649803251713 0
17852343 0
19898168 0
133663801 0
20583993 0
158865339 0
472122299 0
1652541 0
18774524 0
980924206132195328 0
849800022094880768 0
1371529262566608908 0
63485764 0
356631237 0
1283802294186848257 0
149516743 0
96877126 0
21694413 19080
19407053 132939
14946895 122403
16396368 144475
1276154115773599744 429
7313362 223173
2987385810 4298
1232689061049126912 10
596792665 17347
786324746719924224 35246
1260307716385443840 48
1310782718603456513 605
2735591 318440
488554346 478
26329195 532152
23922797 560317
2291106157 0
1303630426234408961 0
15969142 0
19013879 0
1016487483138138112 1772


In [7]:
for user in users:
    print(user, get_cascade_size(user, COLLECTION_NAME, db))

14330880 2
20402945 4451
3191437439 1
1147804652039069696 1
1329524222109097995 70
14500743 1
1282297722364727296 366
9633802 5
1052626822645596161 1
25589776 2
17710740 4
215863060 1
226024854 2
1333137558893719552 1
41085467 1
624413 1
1143770884974399488 1
1295043616742805506 3
295860511 1
353701419 2
1233104313771184129 1
768552674719567876 4
15108530 1
17525171 119
48310197 165
1278251649803251713 1
17852343 2
19898168 1
133663801 2
20583993 1
158865339 98
472122299 145
1652541 4
18774524 1
980924206132195328 1
849800022094880768 1078
1371529262566608908 1
63485764 1
356631237 1
1283802294186848257 2
149516743 59
96877126 93
21694413 1
19407053 5
14946895 1
16396368 1
1276154115773599744 1
7313362 3
2987385810 1
1232689061049126912 1
596792665 23
786324746719924224 3
1260307716385443840 1
1310782718603456513 4
2735591 3
488554346 1
26329195 2
23922797 1
2291106157 2
1303630426234408961 1
15969142 1
19013879 1
1016487483138138112 21
145802745 7
17121146 1
1291779641519169536 1
1479

In [8]:
# Check if the text is same
db[COLLECTION_NAME].distinct('retweet_count')

[0, 1, 2, 3, 4, 5, 6, 7, 16, 25, 62, 80, 106, 125, 126, 185, 406, 778, 1189]

In [9]:
db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 13,
 16,
 25,
 27,
 28,
 31,
 42,
 62,
 80,
 82,
 101,
 106,
 125,
 126,
 142,
 185,
 406,
 461,
 778,
 1126,
 1189,
 2281,
 5302]

In [10]:
db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')

[]

In [11]:
# Check if the text is same
cache = db[COLLECTION_NAME].distinct('retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweet_count':x})['text'])
    print('*'*5)

[126, 185, 406, 778, 1189]
K-Pop superstars BTS have partnered with McDonald’s to offer the band’s favorite order starting May 26 https://t.co/lheFBfHh6e
*****
Fox59 News reported about the upcoming McDonald's BTS Meal and used K-pop Frontman and Face of Kpop #JIMIN's solo p… https://t.co/j0bbcZIqvE
*****
Jimin is truly the face of K-POP as he is the focal point of this  Fox59News piece about the McDonald's BTS Meal!… https://t.co/n4bZ2hIZn8
*****
McDonald’s is teaming up with K-pop megastars @BTS_twt, with a signature meal featuring:
-10-piece Chicken McNugget… https://t.co/TUbZQ1IlZo
*****
Did you hear the news about McDonald's BTS Meal? 😋💜 Wow! The power of this iconic K-pop boy band! 👏👏… https://t.co/w7XDh1njBA
*****


In [12]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.retweet_count':x})['id'])
    print('*'*5)

[778, 1126, 1189, 2281, 5302]
1388896738963083268
*****
1384295838873051136
*****
1384648902796791809
*****
1384296337819070466
*****
1384295798507147264
*****


In [13]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.quoted_status.retweet_count':x})['retweeted_status.quoted_status.id'])
    print('*'*5)

[]


In [14]:
ROOT_USER = 849800022094880768

In [18]:
len(db['followers'].find_one({'user_id': 849800022094880768})['followers'])

21469